In [113]:
import xarray as xr
import numpy as np

from scipy.optimize import fsolve

import datetime

from siphon.catalog import TDSCatalog
from xarray.backends import NetCDF4DataStore

import json
import sys
import logging
import os
import ftplib

<module 'logging' from '/Users/4302001/miniconda3/envs/bound/lib/python3.10/logging/__init__.py'>

In [2]:
logging.basicConfig(format='%(asctime)s [%(levelname)s] %(name)s - %(message)s', 
                    level=logging.INFO, 
                    datefmt='%Y-%m-%d %H:%M:%S')

In [3]:
# for debuggin
import cartopy as cart
import matplotlib.pyplot as plt

Load credentials

In [4]:
def load_credentials(path = "credentials.json"):
    """
    Load CMEMES credentials JSON file. Must contain keys `username` and `password`.
    """
    with open(path, 'r') as cred_file:
        credentials = json.load(cred_file)
    return credentials

In [5]:
cmems_credentials = load_credentials()

## Input

In [6]:
logging.info("test")

2022-05-19 13:37:31 [INFO] root - test


In [7]:
longitude = 317
latitude = 31

dummydate = datetime.datetime(2022, 5, 1)

if longitude < 0:
    print(f"Did you mean `longitude = {360 + longitude}`?")

assert longitude >= 0 and longitude<360, f"`longitude should be between 0 and 360"
assert latitude >= -90 and latitude<=90, "`latitude should be between -90 and 90."

In [9]:
"""
------------------------
PHYSICS HELPER FUNCTIONS
------------------------
"""

'\n------------------------\nPHYSICS HELPER FUNCTIONS\n------------------------\n'

In [10]:
def FSI(T2m, Td2m, T850, W850):
    """
    Computes the Fog Stability Index.
        FSI < 31 indicates a high probability of fog formation, 
        31 < FSI < 55 implies moderate risk of fog
        FSI > 55 suggests low fog risk.
    See https://edepot.wur.nl/144635
        
    Parameters
    ----------
    T2m : int, float
        2 meter temperature
    
    Td2m : int, float
        2 meter dewpoint temperature
    
    T850 : int float
        temperature at 850 hPa
        
    W850 : int, float
        wind speed at 850 hPa
    """
    FSI = 2 * (T2m - Td2m) + 2 * (T2m - T850) + W850
    
    return FSI

In [111]:
def dispersion_relation(kappa, H):
    """
    Computes radial frequency based on wavenumber.
    
    Parameters
    ----------
    kappa : float
        Wavenumber (radial)
    
    H : float
        Locat water depth in meters
    
    Returns
    -------
    float
        frequency
    """
    
    g = 9.81 # m/s^2 gravitational acceleration
    tau = 0.08 # N/m surface tension
    rho = 1026 # kg / m^3 density
    
    sigma = np.sqrt((g * kappa + tau / rho * kappa**3) * np.tanh(kappa * H))
    
    return sigma

In [155]:
def wavelength_velocity(period, water_depth):
    """
    Computes wavelength and phase speed based on the wave period and water depth.
    Assumes free capillary gravity waves.
    
    Parameters
    ----------
    period : float
        Wave period in seconds
    
    water_depth : float
        Local water depth in meters
        
    Returns
    -------
    (float, float)
        (Wavelength, Phase Velocity) tuple
    """
    g = 9.81 # m/s^2 gravitational acceleration
    tau = 0.08 # N/m surface tension
    rho = 1026 # kg / m^3 density
    
    f = 1/period # frequency
    omega = 2 * np.pi * f # radial frequency
    H = water_depth # meters
    
    def find_kappa(x):
        return dispersion_relation(x, H) - omega
    
    for tries in range(1, 10):
        # To vary the starting root of the solver.
        wavenumber = fsolve(find_kappa, tries)[0]
        if wavenumber > 0:
            break
    
    wavelength = 2 * np.pi/wavenumber
    phase_velocity = omega/wavenumber
    
    return wavelength, phase_velocity

In [157]:
"""
------------------------
  DOWNLOADER FUNCTIONS
------------------------
"""

'\n------------------------\n  DOWNLOADER FUNCTIONS\n------------------------\n'

In [12]:
def check_cache():
    if not os.path.isdir("cache"):
        os.mkdir("cache")
        logging.info("Created cache directory.")

In [40]:
def load_gfs_online(lon, lat, time = datetime.datetime.utcnow(), dump_vars = True, margin = 1):
    """
    Load best available GFS model forecast (atmospheric).
    
    Parameters
    ----------
    lon : int or float or (min_lon, max_lon)
        Longitude of location, or longitude bounds of requested box.
        
    lat : int or float or (min_lat, max_lat)
        Latitude of location, or latitude bounds of requested box.
        
    time : datetime.datime
        Timestamp of requested forecast.
        
    dump_vars : bool
        Write a `GFS_vars.json` file. 
        
    margin : int or float
        Margin of data to download around.
        
    Returns
    -------
    xr.dataset
        Dataset with necessary variables
    """
    
    best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/'
                      'Global_0p25deg/catalog.xml?dataset=grib/NCEP/GFS/Global_0p25deg/Best')
    logging.info("Loaded best GFS forecast.")
    
    best_ds = list(best_gfs.datasets.values())[0]
    ncss = best_ds.subset()
    
    if dump_vars:
        with open('available_GFS_vars.txt', 'w') as convert_file:
            for var in ncss.variables:
                convert_file.write(f"{var} \n")
        logging.info("Dumped GFS variables in GFS_vars.json")
    
    # Parse bounds input
    if (type(lon) == float or type(lon) == int) and (type(lat) == float or type(lon) == int):
        min_lon, max_lon = lon - margin, lon + margin
        min_lat, max_lat = lat - margin, lat + margin
    elif len(lon) == 2 and len(lat) == 2:
        min_lon, max_lon = lon
        min_lat, max_lat = lat
    else:
        raise RuntimeError("Error parsing lon and lat input.")
    
    if min_lon < 0 and max_lon < 0:
        west = min_lon + 360
        east = max_lon + 360
        split = False
    elif min_lon < 0 and max_lon >= 0:
        west = min_lon + 360
        east = max_lon
        split = True
    else:
        west, east = min_lon, max_lon
        split = False
        
    logging.info("Parsed input longitude and latitude. Bounding box is: \n"
                f"Lon ({min_lon}, {max_lon}), Lat ({min_lat}, {max_lat})")
    
    logging.info(f"Creating query for time: {time}")
    
    if split:
        logging.info("Splitting query in two boxes")
        query_left, query_right = ncss.query(), ncss.query()
        query_right.lonlat_box(north=max_lat, south=min_lat, west=0, east=east).time(time)
        query_left.lonlat_box(north=max_lat, south=min_lat, west=west, east=360).time(time)

        for query in [query_left, query_right]:
            query.accept('netcdf4')
            # It seems like I can't put these into a dictionary, so letting them be hardcoded for now.
            query.variables("Temperature_surface", 
                            "Low_cloud_cover_low_cloud",
                            "High_cloud_cover_high_cloud",
                            "Medium_cloud_cover_middle_cloud",
                            "Precipitation_rate_surface",
                            "Per_cent_frozen_precipitation_surface",
                            "Categorical_Snow_surface",
                            "Categorical_Rain_surface",
                            "Categorical_Ice_Pellets_surface",
                            "Categorical_Freezing_Rain_surface",
                            "u-component_of_wind_height_above_ground",
                            "v-component_of_wind_height_above_ground",
                            "u-component_of_wind_isobaric",
                            "v-component_of_wind_isobaric",
                            "Dewpoint_temperature_height_above_ground",
                            "Temperature_height_above_ground",
                            "Temperature_isobaric",
                            "Storm_relative_helicity_height_above_ground_layer"
                             )
        logging.info("Sending in query for data.")    
        data_left = ncss.get_data(query_left)
        data_left = xr.open_dataset(NetCDF4DataStore(data_left))
        data_left['lon'] = data_left['lon']-360
        data_right = ncss.get_data(query_right)
        data_right = xr.open_dataset(NetCDF4DataStore(data_right))
        logging.info("Merging left and right data.")
        data = xr.merge([data_left, data_right])
        logging.info("Merging successful.")
    else:
        query = ncss.query()
        query_right.lonlat_box(north=max_lat, south=min_lat, west=west, east=east).time(time)
        query.accept('netcdf4')
        # It seems like I can't put these into a dictionary, so letting them be hardcoded for now.
        query.variables("Temperature_surface", 
                        "Low_cloud_cover_low_cloud",
                        "High_cloud_cover_high_cloud",
                        "Medium_cloud_cover_middle_cloud",
                        "Precipitation_rate_surface",
                        "Per_cent_frozen_precipitation_surface",
                        "Categorical_Snow_surface",
                        "Categorical_Rain_surface",
                        "Categorical_Ice_Pellets_surface",
                        "Categorical_Freezing_Rain_surface",
                        "u-component_of_wind_height_above_ground",
                        "v-component_of_wind_height_above_ground",
                        "u-component_of_wind_isobaric",
                        "v-component_of_wind_isobaric",
                        "Dewpoint_temperature_height_above_ground",
                        "Temperature_height_above_ground",
                        "Temperature_isobaric",
                        "Storm_relative_helicity_height_above_ground_layer"
                         )
        logging.info("Sending in query for data.")
        data = ncss.get_data(query)
        data['lon'] = data['lon'] - 360
    
    logging.info(f"Data succesfully loaded for {data.time.data[0]}, \n"
                 f"initialized at {data.reftime.data[0]}")
    
    return data

In [41]:
CMEMS_id_wave = 'global-analysis-forecast-wav-001-027'
CMEMS_id_phys = 'global-analysis-forecast-phy-001-024'
CMEMS_id_bio = 'global-analysis-forecast-bio-001-028-daily'

In [50]:
def retrieve_copernicus_ftp(path, file, credentials):
    with ftplib.FTP(host = 'my.cmems-du.eu', 
                    user = cmems_credentials['username'], 
                    passwd = cmems_credentials['password']) as ftp:
        ftp.cwd(path)
        with open(file, 'wb') as f:
            ftp.retrbinary('RETR ' + file, f.write)

In [51]:
def download_bathymetry(credentials):
    bathy_server_file = "GLO-MFC_001_030_mask_bathy.nc"
    trimmed_filename = "cache/bathymetry_mask.nc"
    logging.info("Requesting bathymetry file from CMEMS.")
    
    check_cache()
    
    retrieve_copernicus_ftp(path = "Core/GLOBAL_MULTIYEAR_PHY_001_030/cmems_mod_glo_phy_my_0.083-static",
                            file = bathy_server_file,
                            credentials = credentials)
    
    logging.info("Download complete. Trimming unnecessary levels.")
    
    bathy_mask = xr.open_dataset(bathy_server_file).isel(depth = 0)
    
    
    bathy_mask.to_netcdf(trimmed_filename)
    
    if(os.path.isfile(bathy_server_file)):
        os.remove(bathy_server_file)
    
    logging.info(f"Stored bathymetry file as {trimmed_filename}.")

In [72]:
def load_bathymetry():
    bathy_path = "cache/bathymetry_mask.nc"
    check_cache()
    logging.info("Loading bathymetry")
    bathy = xr.open_dataset(bathy_path)
    return bathy

In [53]:
def copernicusmarine_datastore(dataset, username, password):
    from pydap.client import open_url
    from pydap.cas.get_cookies import setup_session
    cas_url = 'https://cmems-cas.cls.fr/cas/login'
    session = setup_session(cas_url, username, password)
    session.cookies.set("CASTGC", session.cookies.get_dict()['CASTGC'])
    database = ['my', 'nrt']
    url = f'https://{database[0]}.cmems-du.eu/thredds/dodsC/{dataset}'
    try:
        data_store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits 
    except:
        url = f'https://{database[1]}.cmems-du.eu/thredds/dodsC/{dataset}'
        data_store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits
    return data_store

In [68]:
def load_cmems_product_online(prod_id, credentials, lon, lat, time = datetime.datetime.utcnow(), dump_vars = True, margin = 1):
    """
    Load best available Copernicus wave model forecast (ocean).
    
    Parameters
    ----------
    credentials : dict
        Dictionary with 'username' and 'password' keys.
    
    lon : int or float or (min_lon, max_lon)
        Longitude of location, or longitude bounds of requested box.
        
    lat : int or float or (min_lat, max_lat)
        Latitude of location, or latitude bounds of requested box.
        
    time : datetime.datime
        Timestamp of requested forecast.
        
    dump_vars : bool
        Write a `GFS_vars.json` file. 
        
    margin : int or float
        Margin of data to download around.
        
    Returns
    -------
    xr.dataset
        Dataset with necessary variables
    """
    logging.info("Logging in to the CMEMS datastore.")
    data_store = copernicusmarine_datastore(prod_id, credentials['username'], credentials['password'])
    dataset = xr.open_dataset(data_store)
    logging.info("Product succesfully loaded.")
    logging.info("Parsing boundaries.")
    # Parse bounds input
    if (type(lon) == float or type(lon) == int) and (type(lat) == float or type(lon) == int):
        min_lon, max_lon = lon - margin, lon + margin
        min_lat, max_lat = lat - margin, lat + margin
    elif len(lon) == 2 and len(lat) == 2:
        min_lon, max_lon = lon
        min_lat, max_lat = lat
    
    lon_array, lat_array = dataset.longitude, dataset.latitude
    min_lon_idx = np.searchsorted(lon_array, min_lon)
    max_lon_idx = np.searchsorted(lon_array, max_lon) + 1
    min_lat_idx = np.searchsorted(lat_array, min_lat)
    max_lat_idx = np.searchsorted(lat_array, max_lat) + 1
    
    logging.info("Subsetting dataset.")
    dataset = dataset.sel(time=time, method='nearest').isel(longitude = slice(min_lon_idx, max_lon_idx + 1),
                                                            latitude = slice(min_lat_idx, max_lat_idx + 1))
    
    return dataset


In [69]:
def load_cmems_wave_data_online(credentials, lon, lat, time = datetime.datetime.utcnow(), margin = 1):
    logging.info("Retrieving CMEMS wave forecast data")
    return load_cmems_product_online(CMEMS_id_wave, credentials, lon, lat, time, margin)

In [70]:
def load_cmems_phys_data_online(credentials, lon, lat, time = datetime.datetime.utcnow(), margin = 1):
    logging.info("Retrieving CMEMS physics forecast and analysis data")
    return load_cmems_product_online(CMEMS_id_phys, credentials, lon, lat, time, margin).isel(depth = 0)

In [71]:
def load_cmems_bio_data_online(credentials, lon, lat, time = datetime.datetime.utcnow(), margin = 1):
    logging.info("Retrieving CMEMS biogeochemistry forecast and analysis data")
    return load_cmems_product_online(CMEMS_id_bio, credentials, lon, lat, time, margin).isel(depth = 0)

In [25]:
"""
------------------------
LOADING THE DATA
------------------------
"""

'\n------------------------\nLOADING THE DATA\n------------------------\n'

In [28]:
atmospheric_data = load_gfs_online((-90, 23), (0, 80))

2022-05-19 13:43:56 [INFO] root - Loaded best GFS forecast.
2022-05-19 13:43:57 [INFO] root - Dumped GFS variables in GFS_vars.json
2022-05-19 13:43:57 [INFO] root - Parsed input longitude and latitude. Bounding box is: 
Lon (-90, 23), Lat (0, 80)
2022-05-19 13:43:57 [INFO] root - Creating query for time: 2022-05-19 11:43:49.395195
2022-05-19 13:43:57 [INFO] root - Splitting query in two boxes
2022-05-19 13:43:57 [INFO] root - Sending in query for data.
2022-05-19 13:44:42 [INFO] root - Merging left and right data.
2022-05-19 13:46:53 [INFO] root - Merging successful.


In [73]:
bathy_data = load_bathymetry()

In [187]:
wave_data = load_cmems_wave_data_online(cmems_credentials, (-90, 23), (0, 80))
phys_data = load_cmems_phys_data_online(cmems_credentials, (-90, 23), (0, 80))
bgc_data = load_cmems_bio_data_online(cmems_credentials, (-90, 23), (0, 80))

In [75]:
"""
------------------------
EXTRACTING THE ENVIRONMENT
------------------------
"""

'\n------------------------\nEXTRACTING THE ENVIRONMENT\n------------------------\n'

In [197]:
class point_conditions:
    def __init__(self, lon, lat):
        """
        Create a JSON-exportable dictionary with the conditions at a specific point on the globe.

        Parameters
        ----------
        lon : int, float
            Longitude, from -180 to 180 degrees

        lat : int, float
            Latitude, from -90 to 90 degrees
        """
        self.lon = lon
        self.lat = lat
        self.conditions = {
                "lon" : {"data" : lon,
                         "description" : "Ranges between -180 and 180.",
                         "units" : "degrees east"},
                "lat" : {"data" : lat,
                         "description" : "Ranges between -180 and 180.",
                         "units" : "degrees east"},
            }
        
        
    @property
    def parameters(self):
        return list(self.conditions.keys())
        
        
    def load_atmospheric_data(self, dataset):
        """
        parameters
        ----------
        atmospheric_data : xr.Dataset
            GFS dataset with conditions that we want to use.
        """
        assert type(dataset) == xr.core.dataset.Dataset
        
        ad_interp = dataset.sel(lon=self.lon, lat=self.lat, method="nearest").isel(time=0)
        
        self.atmospheric_vars = ['clouds_high', 'clouds_middle', 'clouds_low', 
                                 'temperature_2m', 'fog_stability_index', 
                                 'wind_10m_u', 'wind_10m_v', 'wind_10m_speed', 'wind_10m_angle', 
                                 'wind_20m_u', 'wind_20m_v', 'wind_20m_speed', 'wind_20m_angle', 
                                 'precip_rate', 
                                 'precip_rain_cat', 'precip_snow_cat', 'precip_freezing_rain_cat', 
                                 'precip_snow_cat', 'precip_freezing_rain_cat', 'precip_ice_pellets_cat',
                                 'precip_percent_frozen', 'tornadoes']
        
        for var in self.atmospheric_vars:
            self.conditions[var] = {}
        
        self.conditions['clouds_high']['data'] = float(ad_interp["High_cloud_cover_high_cloud"])
        self.conditions['clouds_high']['description'] = ad_interp["High_cloud_cover_high_cloud"].long_name
        self.conditions['clouds_high']['units'] = ad_interp["High_cloud_cover_high_cloud"].units
        
        self.conditions['clouds_middle']['data'] = float(ad_interp["Medium_cloud_cover_middle_cloud"])
        self.conditions['clouds_middle']['description'] = ad_interp["Medium_cloud_cover_middle_cloud"].long_name
        self.conditions['clouds_middle']['units'] = ad_interp["Medium_cloud_cover_middle_cloud"].units
        
        self.conditions['clouds_low']['data'] = float(ad_interp["Low_cloud_cover_low_cloud"])
        self.conditions['clouds_low']['description'] = ad_interp["Low_cloud_cover_low_cloud"].long_name
        self.conditions['clouds_low']['units'] = ad_interp["Low_cloud_cover_low_cloud"].units

        self.conditions['temperature_2m']['data'] = float(ad_interp.Temperature_height_above_ground.sel(height_above_ground5 = 2)) - 273.15 # Celcius
        self.conditions['temperature_2m']['description'] = "Temperature @ 2 meters height"
        self.conditions['temperature_2m']['units'] = "Degrees Celcius"

        self.conditions['fog_stability_index']['data'] = FSI(self.conditions['temperature_2m']['data'],
                                                    float(ad_interp.Dewpoint_temperature_height_above_ground) - 273.15,
                                                    float(ad_interp.Temperature_isobaric.sel(isobaric1 = 85000)) - 273.15,
                                                    np.sqrt(float(ad_interp["u-component_of_wind_isobaric"].sel(isobaric1 = 85000))**2 + \
                                                            float(ad_interp["v-component_of_wind_isobaric"].sel(isobaric1 = 85000))**2)
                                                   )
        self.conditions['fog_stability_index']['description'] = "Computes the Fog Stability Index. \n" +\
                                                           "     FSI < 31 indicates a high probability of fog formation \n" +\
                                                           "     31 < FSI < 55 implies moderate risk of fog \n" +\
                                                           "     FSI > 55 suggests low fog risk. \n" +\
                                                           "See https://edepot.wur.nl/144635"
        self.conditions['fog_stability_index']['units'] = ""
        
        self.conditions['wind_10m_u']['data'] = float(ad_interp["u-component_of_wind_height_above_ground"].sel(height_above_ground4 = 10))
        self.conditions['wind_10m_u']['description'] = "u-component of wind at 10 meters height"
        self.conditions['wind_10m_u']['units'] = ad_interp["u-component_of_wind_height_above_ground"].units
        
        self.conditions['wind_10m_v']['data'] = float(ad_interp["v-component_of_wind_height_above_ground"].sel(height_above_ground4 = 10))
        self.conditions['wind_10m_v']['description'] = "v-component of wind at 10 meters height"
        self.conditions['wind_10m_v']['units'] = ad_interp["v-component_of_wind_height_above_ground"].units
        
        self.conditions['wind_10m_speed']['data'] = np.sqrt(self.conditions['wind_10m_u']['data']**2 + self.conditions['wind_10m_v']['data']**2)
        self.conditions['wind_10m_speed']['description'] = "Wind speed at 10 meters height."
        self.conditions['wind_10m_speed']['units'] = "m/s"
        
        self.conditions['wind_10m_angle']['data'] = np.arctan2(self.conditions['wind_10m_u']['data'], self.conditions['wind_10m_v']['data'])/np.pi*180 # 
        self.conditions['wind_10m_angle']['description'] = "Wind angle at 10 meters heights."
        self.conditions['wind_10m_angle']['units'] = "Degrees with respect to the north. Negative values are on the westward side of the compass."
        
        self.conditions['wind_20m_u']['data'] = float(ad_interp["u-component_of_wind_height_above_ground"].sel(height_above_ground4 = 20))
        self.conditions['wind_20m_u']['description'] = "u-component of wind at 20 meters height"
        self.conditions['wind_20m_u']['units'] = ad_interp["u-component_of_wind_height_above_ground"].units
        
        self.conditions['wind_20m_v']['data'] = float(ad_interp["v-component_of_wind_height_above_ground"].sel(height_above_ground4 = 20))
        self.conditions['wind_20m_v']['description'] = "v-component of wind at 20 meters height"
        self.conditions['wind_20m_v']['units'] = ad_interp["v-component_of_wind_height_above_ground"].units
        
        self.conditions['wind_20m_speed']['data'] = np.sqrt(self.conditions['wind_20m_u']['data']**2 + self.conditions['wind_20m_v']['data']**2)
        self.conditions['wind_20m_speed']['description'] = "Wind speed at 20 meters heigh.t"
        self.conditions['wind_20m_speed']['units'] = "m/s"
        
        self.conditions['wind_20m_angle']['data'] = np.arctan2(self.conditions['wind_20m_u']['data'], self.conditions['wind_20m_v']['data'])/np.pi*180 # 
        self.conditions['wind_20m_angle']['description'] = "Wind angle at 20 meters height."
        self.conditions['wind_20m_angle']['units'] = "Degrees with respect to the north. Negative values are on the westward side of the compass."

        self.conditions['precip_rate']['data'] = float(ad_interp.Precipitation_rate_surface)
        self.conditions['precip_rate']['description'] = ad_interp["Precipitation_rate_surface"].long_name
        self.conditions['precip_rate']['units'] = ad_interp["Precipitation_rate_surface"].units
        
        self.conditions['precip_rain_cat']['data'] = float(ad_interp.Categorical_Rain_surface)
        self.conditions['precip_rain_cat']['description'] = ad_interp["Categorical_Rain_surface"].long_name
        self.conditions['precip_rain_cat']['units'] = "0 = No, 1 = Yes. See https://www.nco.ncep.noaa.gov/pmb/docs/grib2/grib2_doc/grib2_table4-222.shtml"
        
        self.conditions['precip_snow_cat']['data'] = float(ad_interp.Categorical_Snow_surface)
        self.conditions['precip_snow_cat']['description'] = ad_interp["Categorical_Snow_surface"].long_name
        self.conditions['precip_snow_cat']['units'] = "0 = No, 1 = Yes. See https://www.nco.ncep.noaa.gov/pmb/docs/grib2/grib2_doc/grib2_table4-222.shtml"
        
        self.conditions['precip_freezing_rain_cat']['data'] = float(ad_interp.Categorical_Freezing_Rain_surface)
        self.conditions['precip_freezing_rain_cat']['description'] = ad_interp["Categorical_Freezing_Rain_surface"].long_name
        self.conditions['precip_freezing_rain_cat']['units'] = "0 = No, 1 = Yes. See https://www.nco.ncep.noaa.gov/pmb/docs/grib2/grib2_doc/grib2_table4-222.shtml"
        
        self.conditions['precip_ice_pellets_cat']['data'] = float(ad_interp.Categorical_Ice_Pellets_surface)
        self.conditions['precip_ice_pellets_cat']['description'] = ad_interp["Categorical_Ice_Pellets_surface"].long_name
        self.conditions['precip_ice_pellets_cat']['units'] = "0 = No, 1 = Yes. See https://www.nco.ncep.noaa.gov/pmb/docs/grib2/grib2_doc/grib2_table4-222.shtml"
        
        self.conditions['precip_percent_frozen']['data'] = float(ad_interp.Per_cent_frozen_precipitation_surface)
        self.conditions['precip_percent_frozen']['description'] = ad_interp["Per_cent_frozen_precipitation_surface"].long_name
        self.conditions['precip_percent_frozen']['units'] = ad_interp["Per_cent_frozen_precipitation_surface"].units
        
        self.conditions['tornadoes']['data'] = float(ad_interp["Storm_relative_helicity_height_above_ground_layer"] > 250)
        self.conditions['tornadoes']['description'] = ad_interp["Storm_relative_helicity_height_above_ground_layer"].long_name + \
                                                                "\n H" + \
                                                                "\n See https://www.spc.noaa.gov/exper/mesoanalysis/help/help_srh1.html"
        self.conditions['tornadoes']['units'] = "1 if higher than 250 J/kg (tornados may develop). 0 if lower."
    

    def load_bathy_data(self, dataset):
        """
        parameters
        ----------
        atmospheric_data : xr.Dataset
            Bathymetry dataset that we want to use.
        """
        assert type(dataset) == xr.core.dataset.Dataset
        
        batd_interp = dataset.sel(longitude=self.lon, latitude=self.lat, method="nearest").compute()
        
        self.conditions['bathy'] = {"data" : float(batd_interp.deptho),
                                    "description" : "Seafloor depth",
                                    "units" : "m"}
        self.depth = self.conditions['bathy']['data']
        self.conditions['mask'] = {"data" : int(batd_interp.mask),
                                    "description" : "Land-sea mask",
                                    "units" : "1 = sea ; 0 = land"}
        
        
    def load_wave_data(self, dataset):
        """
        parameters
        ----------
        atmospheric_data : xr.Dataset
            CMEMS wave dataset with conditions that we want to use.
        """
        assert type(dataset) == xr.core.dataset.Dataset
        if not 'bathy' in self.conditions:
            logging.warning("Bathymetric data is necessary for determining wave parameters. Wavelength cannot be computed.")
        waved_interp = dataset.sel(longitude=self.lon, latitude=self.lat, method="nearest").compute()
        
        for variable in list(waved_interp.variables):
            if variable not in ['longitude', 'latitude', 'time']:
                logging.info(f"Downloading variable {variable}")
                self.conditions[variable] = {"data" : float(waved_interp[variable]),
                                             "description" : waved_interp[variable].long_name,
                                             "units" : waved_interp[variable].units}
        
        peak_wavelength, peak_velocity = wavelength_velocity(self.conditions['VTPK']['data'], self.depth)
        self.conditions['VLPK'] = {"data" : peak_wavelength,
                                 "description" : "Wavelength of wave at spectral peak",
                                 "units" : "m"}
        self.conditions['VSPK'] = {"data" : peak_velocity,
                                 "description" : "Speed of wave at spectral peak",
                                 "units" : "m/s"}
        
        mean_wavelength, mean_velocity = wavelength_velocity(self.conditions['VTM10']['data'], self.depth)
        self.conditions['VLM0110'] = {"data" : mean_wavelength,
                                 "description" : "mean wavelength from variance spectral density",
                                 "units" : "m"}
        self.conditions['VSM0110'] = {"data" : mean_velocity,
                                 "description" : "mean speed from variance spectral density",
                                 "units" : "m/s"}
        
        SW1_wavelength, SW1_velocity = wavelength_velocity(self.conditions['VTM01_SW1']['data'], self.depth)
        self.conditions['VLM01_SW1'] = {"data" : SW1_wavelength,
                                 "description" : "Spectral moments (0,1) primary swell wave wavelength",
                                 "units" : "m"}
        self.conditions['VSM01_SW1'] = {"data" : SW1_velocity,
                                 "description" : "Spectral moments (0,1) primary swell wave speed",
                                 "units" : "m/s"}
        
        SW2_wavelength, SW2_velocity = wavelength_velocity(self.conditions['VTM01_SW2']['data'], self.depth)
        self.conditions['VLM01_SW2'] = {"data" : SW1_wavelength,
                                 "description" : "Spectral moments (0,1) secondary swell wave wavelength",
                                 "units" : "m"}
        self.conditions['VSM01_SW2'] = {"data" : SW1_velocity,
                                 "description" : "Spectral moments (0,1) secondary swell waved speed",
                                 "units" : "m/s"}
        
        WW_wavelength, WW_velocity = wavelength_velocity(self.conditions['VTM01_WW']['data'], self.depth)
        self.conditions['VLM01_WW'] = {"data" : WW_wavelength,
                                 "description" : "Spectral moments (0,1) wind wave wavelength",
                                 "units" : "m"}
        self.conditions['VSM01_WW'] = {"data" : WW_velocity,
                                 "description" : "Spectral moments (0,1) wind wave spee",
                                 "units" : "m/s"}
        
        
    def load_phys_data(self, dataset):
        """
        parameters
        ----------
        dataset : xr.Dataset
            CMEMS physics dataset with conditions that we want to use.
        """
        assert type(dataset) == xr.core.dataset.Dataset
        
        physd_interp = dataset.sel(longitude=self.lon, latitude=self.lat, method="nearest")
        for variable in list(physd_interp.variables):
            if variable in ['sithick', 'siconc', 'thetao', 'uo', 'vo', 'vsi', 'usi']:
                logging.info(f"Downloading variable {variable}")
                self.conditions[variable] = {"data" : float(physd_interp[variable]),
                                             "description" : physd_interp[variable].long_name,
                                             "units" : physd_interp[variable].units}
        
    def load_bgc_data(self, dataset):
        """
        parameters
        ----------
        dataset : xr.Dataset
            CMEMS biogeochemistry dataset with conditions that we want to use.
        """
        assert type(dataset) == xr.core.dataset.Dataset
        
        bgcd_interp = dataset.sel(longitude=self.lon, latitude=self.lat, method="nearest") 
        for variable in list(bgcd_interp.variables):
            if variable in ['nppv', 'chl']:
                logging.info(f"Downloading variable {variable}")
                self.conditions[variable] = {"data" : float(bgcd_interp[variable]),
                                             "description" : bgcd_interp[variable].long_name,
                                             "units" : bgcd_interp[variable].units}

In [198]:
conditions = point_conditions(-30, 30)
conditions.load_atmospheric_data(atmospheric_data)
conditions.load_bathy_data(bathy_data)
conditions.load_wave_data(wave_data)
conditions.load_phys_data(phys_data)
conditions.load_bgc_data(bgc_data)

In [188]:
list(bgc_data.variables)

['longitude',
 'latitude',
 'depth',
 'time',
 'chl',
 'fe',
 'no3',
 'nppv',
 'o2',
 'ph',
 'phyc',
 'po4',
 'si',
 'spco2']

In [192]:
bgc_data.nppv

<xarray.DataArray 'nppv' (latitude: 322, longitude: 454)>
[146188 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 -90.0 -89.75 -89.5 ... 22.75 23.0 23.25
  * latitude   (latitude) float32 0.0 0.25 0.5 0.75 ... 79.5 79.75 80.0 80.25
    depth      float32 0.494
    time       datetime64[ns] 2022-05-19T12:00:00
Attributes:
    long_name:      Total Primary Production of Phyto
    standard_name:  net_primary_production_of_biomass_expressed_as_carbon_per...
    units:          mg m-3 day-1
    unit_long:      milligrams of Carbon per cubic meter per day
    valid_min:      0.0
    valid_max:      1085.8016
    _ChunkSizes:    [1, 13, 171, 360]

In [191]:
bgc_data.chl.min()

<xarray.DataArray 'chl' ()>
array(0.01639749, dtype=float32)
Coordinates:
    depth    float32 0.494
    time     datetime64[ns] 2022-05-19T12:00:00

In [172]:
load_bathymetry(cmems_credentials)

In [204]:
atmospheric_data.sel(lon = -30, lat= 30)['Categorical_Rain_surface'].long_name

'Categorical Rain @ Ground or water surface'

In [165]:
list(wave_data.variables)

['longitude',
 'latitude',
 'time',
 'VHM0',
 'VMDR_WW',
 'VHM0_WW',
 'VMDR_SW1',
 'VTM01_SW1',
 'VMDR_SW2',
 'VTM01_SW2',
 'VMDR',
 'VTM10',
 'VHM0_SW1',
 'VHM0_SW2',
 'VTPK',
 'VSDX',
 'VSDY',
 'VPED',
 'VTM02',
 'VTM01_WW']

In [ ]:
wave_data.VTM01_WW

<xarray.DataArray 'VTM01_WW' (latitude: 962, longitude: 1358)>
[1306396 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float64 -90.0 -89.92 -89.83 ... 22.92 23.0 23.08
  * latitude   (latitude) float64 0.0 0.08333 0.1667 0.25 ... 79.92 80.0 80.08
    time       datetime64[ns] 2022-05-19T12:00:00
Attributes:
    long_name:         Spectral moments (0,1) wind wave period
    units:             s
    standard_name:     sea_surface_wind_wave_mean_period
    cell_methods:      time:point area:mean
    type_of_analysis:  spectral analysis
    WMO_code:          223
    _ChunkSizes:       [1, 681, 1440]